In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "college-information-llm"

In [2]:
llm = ChatOpenAI()

In [3]:
from knowledgebase import TXTKnowledgeBase

In [4]:
kb=TXTKnowledgeBase(txt_source_folder_path='lxbd')

In [5]:
#kb.initiate_documents()

In [6]:
vector=kb.return_retriever_from_persistant_vector_db()
retriever = vector.as_retriever(search_kwargs={'k':4})

In [7]:
from langchain.tools.retriever import create_retriever_tool

In [72]:
retriever_tool=create_retriever_tool(
    retriever,
    name='search_international_students_related_information',
    description='搜索并返回关于在美国留学相关的信息',
)

another tools

In [73]:
from langchain_core.tools import tool


@tool
def college_data(collegeID:int):
    """搜索美国大学相关的各项数据并作图"""
    print(collegeID)
    return

In [74]:
tools=[retriever_tool,college_data]

In [75]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [76]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [80]:
result = agent_executor.invoke({"input": "普林斯顿大学"})

1


In [81]:
result

{'input': '普林斯顿大学', 'output': '抱歉，我无法找到有关普林斯顿大学的数据。您还有其他问题需要帮忙吗？'}